## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '..')

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from dotenv import load_dotenv

load_dotenv()

import json

In [7]:
from loguru import logger

import os

## Load the index

In [9]:
DATA_PATH = os.getenv('DATA_PATH')
storage_context_fp = os.path.join(DATA_PATH, 'storage_context')

In [38]:
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.vector_stores.opensearch import (
    OpensearchVectorClient,
    OpensearchVectorStore,
)

dim = 768
client = OpensearchVectorClient(
    endpoint=os.getenv('OPENSEARCH_ENDPOINT', 'http://localhost:9200'),
    index=os.getenv('OPENSEARCH_INDEX', 'index-demo'),
    dim=dim,
)
vector_store = OpensearchVectorStore(client)

logger.info(f"Loading Storage Context from {storage_context_fp}...")
docstore = SimpleDocumentStore.from_persist_dir(persist_dir=storage_context_fp)
storage_context = StorageContext.from_defaults(
    docstore=docstore,
    vector_store=vector_store
)
nodes = list(docstore.docs.values())

2024-11-17 02:15:41.962 | INFO     | __main__:<module>:17 - Loading Storage Context from data/storage_context...


## Setting LLM

In [25]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

model_name = "BookingCare/gte-multilingual-base-v1"
embed_model = HuggingFaceEmbedding(model_name, trust_remote_code=True)

In [26]:
from llama_index.core.settings import Settings
Settings.embed_model = embed_model

## Contruct Retriever

In [36]:
from llama_index.core.response.notebook_utils import display_source_node

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

## Vector search

In [28]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
top_k = 10

vector_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode='mmr',
    similarity_top_k=top_k
)

In [39]:
results = vector_retriever.retrieve("bệnh tiểu đường?")
for result in results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

**Node ID:** 0ca78880-3b22-418f-b278-2c9e6ccc58da<br>**Similarity:** 0.64239794<br>**Text:** Các loại bệnh tiểu đường.
Hiện nay, bệnh tiểu đường được phân thành ba loại chính, bao gồm: Bệnh tiểu đường loại 1 : Đây là một bệnh rối loạn tự miễn, nghĩa là hệ thống miễn dịch của cơ thể bị nhầm lẫn và tấn công chính tuyến tụy. Điều này làm cản trở quá trình sản xuất insulin và dẫn đến thiếu hụt đáng kể số lượng insulin cần thiết cho các hoạt động chuyển hóa glucose thành nguồn năng lượng của cơ thể, từ đó khiến cho lượng đường trong máu tăng cao. Bệnh tiểu đường loại 1 thường khá hiếm gặp, và xảy ra chủ yếu ở trẻ nhỏ. Bệnh tiểu đường loại 2 : Đây là loại tiểu đường thường gặp nhất hiện nay, ảnh hưởng lớn đến những người bị béo phì. Khi mắc bệnh tiểu đường loại 2, cơ thể sẽ bị đề kháng với insulin. Mặc dù hormone insulin vẫn được tiết ra bởi tuyến tụy, nhưng nó không thể chuyển hóa được đường glucose, khiến đường tích tụ dần trong máu. Hầu hết các trường hợp bị tiểu đường đều mắc phải loại tiểu đường này. Bệnh tiểu đường thai kỳ : Thường xảy ra đối với phụ nữ đang mang thai. Nhìn...<br>**Metadata:** {}<br>

**Node ID:** 309fb63d-648f-40a1-ab8f-b63c19126e31<br>**Similarity:** 0.6154264<br>**Text:** Nguyên nhân và triệu chứng gây nên tiền đái tháo đường.
Insulin - hormone được tạo ra bởi tuyến tụy giúp cơ thể bạn chuyển hóa glucose thành nhiên liệu. Khi ăn thức ăn, carbohydrate trong thức ăn của bạn được chuyển hóa thành glucose , hay còn gọi là đường huyết. Glucose đó sẽ ở trong máu cho đến khi tuyến tụy tiết ra insulin. Insulin hoạt động giống như một chiếc chìa khóa mở các tế bào để cho phép glucose đi vào, nơi nó được sử dụng để cung cấp năng lượng cho cơ thể. Nếu không có insulin hoặc khi insulin không hoạt động hiệu quả, lượng glucose đó sẽ bị mắc kẹt trong máu và tích tụ, khiến lượng đường trong máu tăng lên. Trong bệnh đái tháo đường loại 2, tuyến tụy của bạn vẫn có thể sản xuất insulin, nhưng insulin đó dần trở nên kém hiệu quả hơn trong việc giúp glucose vào tế bào của bạn. Tiền đái tháo đường là một dấu hiệu cho thấy cơ thể bạn đang bắt đầu ngừng sử dụng insulin một cách hiệu quả nhất. Nói cách khác, cơ thể đang bắt đầu đề kháng với insulin. Khi cơ thể chống lại insu...<br>**Metadata:** {}<br>

**Node ID:** f5083c8b-80db-4eda-a661-a70b43c06697<br>**Similarity:** 0.5889128<br>**Text:** Khái niệm hạ đường huyết.
Hạ đường huyết là những cơn đường huyết thấp bất thường có hoặc không có triệu chứng. Đường huyết ≤ 70 mg/dL (3.9 mmol/L) (Đường huyết tĩnh mạch chính xác hơn nhưng cũng có thể chấp nhận đường huyết mao mạch) Đái tháo đường typ 1 hay bị hạ đường huyết ở những bệnh nhân điều trị quá tích cực. Có 2- 4 % trường hợp tử vong liên quan hạ đường huyết. Đái tháo đường typ 2, tỷ lệ hạ đường huyết liên quan đến thuốc hạ đường huyết đang dùng: Sulfonylure, Insulin,..<br>**Metadata:** {}<br>

**Node ID:** 5d138941-b47f-4527-a07b-e598d946a534<br>**Similarity:** 0.5860211<br>**Text:** Đái tháo đường.
Đái tháo đường là bệnh rối loạn chuyển hóa không đồng nhất, làm tăng glucose máu. Nguyên nhân dẫn tới bệnh đái tháo đường là do khiếm khuyết về tiết insulin, hoặc tác động của insulin hoặc cả hai. Tăng glucose mạn tính kéo dài sẽ gây ra những rối loạn chuyển hóa protide, lipid, carbohydrate,... Sau đó dẫn tới tổn thương nhiều đến cơ quan khác nhau đặc biệt là mạch máu, thần kinh, tim mạch, mắt, thần kinh. Các loại đái tháo đường bao gồm: Đái tháo đường typ 1 : Hay còn gọi là đái tháo đường phụ thuộc insulin, do phá hủy tế bào beta tụy dẫn tới thiếu insulin tuyệt đối. Đái tháo đường typ 2 : Hay đái tháo đường không phụ thuộc insulin, do giảm chức năng của tế bào beta tụy tiến triển trên nền tảng đề kháng insulin. Đái tháo đường thai kỳ : Được chẩn đoán trong 3 tháng giữa hoặc 3 tháng cuối của thai kỳ. Trước đó bệnh nhân không mắc đái tháo đường typ 1, typ 2. Đái tháo đường do nguyên nhân khác: đái tháo đường do dùng thuốc, đái tháo đường sơ sinh,... Trong đó, đái tháo...<br>**Metadata:** {}<br>

**Node ID:** 1009171b-32b0-4f6c-9594-02ab0c3ff661<br>**Similarity:** 0.56146365<br>**Text:** Tại sao đường huyết lại dao động, không ổn định.
Trước khi tìm hiểu chỉ số đường huyết bao nhiêu là nguy hiểm, bạn đã biết tại sao đường huyết lại dao động, không ổn định hay chưa? Theo đó, chỉ số đường huyết vốn được hiểu là nồng độ glucose (đường) trong máu với đơn vị đo là mmol/L hoặc mg/dL. Trường hợp chỉ số đường huyết dao động không ổn định, dù quá thấp hoặc quá cao cũng đều sẽ gây ra những ảnh hưởng tiêu cực tới sức khỏe người bệnh. Các chuyên gia cho biết, có 4 nguyên nhân chính làm chỉ số đường huyết bất ổn gồm: Chế độ ăn uống của người bệnh không hợp lý, thiếu khoa học. Thường xuyên chịu áp lực, stress, căng thẳng trong cuộc sống. Sinh hoạt không điều độ, thường xuyên mất ngủ hoặc thiếu ngủ Sử dụng một số loại thuốc bao gồm thuốc kháng viêm, thuốc tránh thai, thuốc dị ứng, thuốc chống trầm cảm ,...<br>**Metadata:** {}<br>

**Node ID:** 60a15fc1-5e39-418c-a6ca-c8c54b8e0521<br>**Similarity:** 0.5509595<br>**Text:** Bệnh đái tháo đường có phải uống thuốc suốt đời không?.
Nếu bạn đang dùng thuốc metformin , đây là thuốc đầu tay cho các bác sỹ lâm sàng nội tiết trong phác đồ điều trị phổ biến cho bệnh đái tháo đường type 2 . Sau một thời gian bác sĩ có thể giảm liều thuốc metformin dần theo từng giai đoạn khi bạn đã giảm cân và đạt được trọng lượng lý tưởng. Thậm chí người bệnh có thể ngừng hẳn một thời gian không cần sử dụng metformin nếu bạn đang điều chỉnh lối sống lành mạnh và kiểm soát tốt được lượng đường huyết trong máu bằng liều thấp trong vài tháng. Sau đó, bác sĩ sẽ theo dõi sát và hẹn tái khám lại người bệnh nếu thử ngưng sử dụng thuốc hoặc sử dụng liều thấp hơn. Tuy nhiên vẫn còn một số lượng bệnh nhân khi người bệnh đã cố gắng kiểm soát đường máu bằng điều chỉnh chế độ dinh dưỡng và tập luyện thể dục thì vẫn có thể phải quay lại dùng thuốc. Nguyên nhân là đái tháo đường là căn bệnh không thể điều trị khỏi mà tiến triển càng ngày càng nặng hơn. Một nghiên cứu cho thấy những người mắc ...<br>**Metadata:** {}<br>

**Node ID:** ed7c0b91-d647-45f5-8d6a-ee57c421228d<br>**Similarity:** 0.54465806<br>**Text:** Làm thế nào để cải thiện giấc ngủ ở những người mắc bệnh tiểu đường?.
Khác với tình trạng bệnh tiểu đường ngủ nhiều hay người tiểu đường ngủ li bì . Trường hợp người mắc bệnh bị mất ngủ, bệnh nhân mắc tiểu đường có thể thực hiện theo một số phương pháp sau đây để cải thiện chất lượng giấc ngủ của mình, bao gồm: 4.1 Hạn chế sử dụng các thiết bị điện tử trước giờ đi ngủ Bạn không nên sử dụng các thiết bị có ánh sáng xanh vào ban đêm trước khi đi ngủ, chẳng hạn như điện thoại hoặc máy tính, vì những thiết bị này có thể kích thích não bộ hoạt động và khiến bạn khó đi vào giấc ngủ hơn. Thay vào đó, có thể đọc một vài trang sách giấy trước giờ đi ngủ để đầu óc được thư giãn và bớt bị mỏi mắt hơn. 4.2 Không sử dụng đồ uống có cồn trước khi đi ngủ Mặc dù việc uống một ly rượu có thể giúp bạn cảm thấy muốn ngủ hơn, tuy nhiên khi tiêu thụ đồ uống có cồn sẽ khiến bạn khó có thể ngủ sâu và đủ giấc được. 4.3 Thiết lập giờ đi ngủ cố định Bạn nên đi ngủ vào một khung giờ nhất định trong ngày, kể c...<br>**Metadata:** {}<br>

**Node ID:** e40711fe-41db-4580-a095-0aa4f8c90819<br>**Similarity:** 0.5298098<br>**Text:** Chế độ tập luyện cho bệnh nhân đái tháo đường phù hợp nhất.
Các bác sĩ chuyên khoa Nội tiết cũng như hội nội tiết đái tháo đường đều đưa ra khuyến nghị các hoạt động thể chất mang lại hiệu quả khi đạt được tối thiểu các tiêu chuẩn sau đây: Ít nhất hai tiếng rưỡi hoạt động với cường độ trung bình đến mạnh trong mỗi tuần như là đi bộ nhanh, thể dục nhịp điệu, bơi lội hoặc chạy bộ. Tập thể dục từ hai đến ba buổi đối kháng mỗi tuần giúp tăng cường sức mạnh cơ bắp như các động tác nâng tạ tay hoặc tập chống đẩy. Không để nên quá hai ngày liên tiếp mà không có bất kỳ hoạt động thể chất nào. Không nên ngồi lâu liên tục quá 30 phút trong ngày. Kết hợp các bài tập linh hoạt với nhau như những thói quen hay sở thích của riêng mình.<br>**Metadata:** {}<br>

**Node ID:** 11a61dda-8717-4257-a784-a64cd3713046<br>**Similarity:** 0.5234243<br>**Text:** Chăm sóc những vết bầm tím cẩn thận.
Tiểu đường làm tăng nguy cơ nhiễm trùng và làm chậm quá trình liền thương , vì vậy hãy nhanh chóng điều trị ngay cả những vết cắt và vết xước đơn giản. Vệ sinh vết thương đúng cách và sử dụng kháng sinh, băng vô trùng. Đi khám bác sĩ ngay nếu tình trạng không khá hơn trong vài ngày. Kiểm tra bàn chân mỗi ngày để xác định các vết phồng rộp, vết cắt, vết loét, mẩn đỏ hoặc sưng tấy. Bạn nên giữ ẩm cho bàn chân ngăn ngừa các vết nứt xảy ra và vết thương. Bàn chân đái tháo đường là một biến chứng bệnh tiểu đường nghiêm trọng. Tuy nhiên, nếu được chẩn đoán sớm và điều trị kịp thời thì sẽ cho việc điều trị ít xâm lấn và hiệu quả hơn.<br>**Metadata:** {}<br>

**Node ID:** dba688d6-9e2f-437e-9180-2ddfd9bd89bf<br>**Similarity:** 0.51702386<br>**Text:** Bệnh tiểu đường có nên được khuyến nghị sử dụng nước mía không?.
Giống như các loại đồ uống nhiều đường khác, nước mía là một lựa chọn không tồi nếu bạn bị tiểu đường. Tuy nhiên, lượng đường “khổng lồ” chứa trong nước mía có thể làm tăng lượng đường trong máu của cơ thể một cách nguy hiểm. Vì vậy, bạn nên tránh hoàn toàn đồ uống này để hạn chế những ảnh hưởng tiêu cực nó có thể mang lại cho cơ thể. Tiểu đường uống nước mía, nên hay không? Các nghiên cứu trong ống nghiệm về chiết xuất mía cho thấy rằng, chất chống oxy hóa polyphenol của nước mía có thể giúp các tế bào tuyến tụy sản xuất nhiều insulin hơn - hormone điều chỉnh lượng đường trong máu của cơ thể. Tuy vậy, các nghiên cứu này chỉ là sơ bộ và không an toàn cho những người mắc bệnh tiểu đường . Nếu bạn vẫn thích uống ngọt, bạn có thể dùng trái cây tươi để tạo vị ngọt tự nhiên. Nước mía là thức uống không tinh chế được chiết xuất từ ​​cây mía. Mặc dù nó cung cấp một lượng chất chống oxy hóa lành mạnh nhưng nó lại chứa rất nhiề...<br>**Metadata:** {}<br>

## Sparse retrieval using BM25

In [42]:
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever(
    nodes=nodes,
    similarity_top_k=top_k,
)

In [43]:
results = bm25_retriever.retrieve("bệnh tiểu dường?")
for result in results:
    display_source_node(result, source_length=1000, show_source_metadata=True)


**Node ID:** 1c01da7d-59fc-479b-b0d7-1b7c6411cfda<br>**Similarity:** 2.748440980911255<br>**Text:** Đưa bé đi điều trị răng khi răng bị mẻ.
Nếu răng và nướu của trẻ trông ổn và dường như không bị đau, trẻ sẽ không sao mà không cần kiểm tra răng miệng. Nếu răng bị mẻ hoặc nứt và bé có vẻ bị đau, bạn nên đưa bé đến nha sĩ ngay vì một phần của dây thần kinh có thể bị tổn thương. Bạn cũng nên đưa bé đi khám răng nếu răng lung lay nhiều. Bác sĩ sẽ xem cho bé một chiếc răng có vẻ như không đúng vị trí. Đánh giá xem nó có cần được định vị lại hay không. Nếu răng bị sứt mẻ dường như không gây phiền toái cho bé, hãy đặt lịch hẹn với nha sĩ để đánh giá xem có vết nứt bên dưới hoặc tổn thương khác mà bạn không thể nhìn thấy hay không. Bác sĩ sẽ sửa răng bằng cách dũa hoặc hàn nó bằng vật liệu kết dính.<br>**Metadata:** {}<br>

**Node ID:** a2f21c64-7821-4c63-8a0f-898db5908edc<br>**Similarity:** 2.5576696395874023<br>**Text:** Đôi nét về bệnh ung thư vú.
Ung thư vú là loại ung thư phổ biến thứ 2 ở phụ nữ, chỉ đứng sau ung thư da . Mặc dù tình trạng này có thể xảy ra ở cả nam giới, tuy nhiên rất hiếm gặp. Theo các nhà nghiên cứu, nguyên nhân chính xác dẫn đến bệnh ung thư vú vẫn chưa được biết rõ, nhưng dường như nguy cơ phát triển căn bệnh này có xu hướng tăng lên theo tuổi tác. Nhìn chung, khả năng mắc ung thư vú đặc biệt cao ở những phụ nữ trên 60 tuổi. Chính vì có tỷ lệ mắc bệnh khi tuổi tác ngày một cao, cho nên những phụ nữ này sẽ dễ gặp phải tình trạng loãng xương do ung thư vú.<br>**Metadata:** {}<br>

**Node ID:** 6dafade0-cb10-494d-b099-c4eabc95a9f7<br>**Similarity:** 2.400062084197998<br>**Text:** Có cách nào để ngăn ngừa bệnh viêm amidan không?.
Nếu trẻ dễ bị viêm họng , việc giữ gìn sức khỏe cho trẻ dường như là một cuộc chiến trường kỳ, đặc biệt là khi trẻ đã đi học. Bạn cần quan tâm trẻ kỹ hơn. Trẻ cần được rửa tay thường xuyên và không dùng chung đồ dùng ăn uống với người khác. Điều này có thể làm giảm đáng kể số ca nhiễm trùng mà trẻ em mắc phải. Tăng cường khả năng miễn dịch tốt có thể cho trẻ. Cho trẻ ăn một chế độ ăn uống cân bằng và đảm bảo rằng chúng được hít thở không khí trong lành, tập thể dục thường xuyên và dành nhiều thời gian để nghỉ ngơi. Nguồn tham khảo: babycenter.com<br>**Metadata:** {}<br>

**Node ID:** ace813e8-d4b1-4663-b637-ed6da2e57075<br>**Similarity:** 2.2394800186157227<br>**Text:** Thay đổi tính cách.
Tính cách là tất cả những gì một người suy nghĩ, cảm nhận và hành động. Đó cũng có thể là thói quen, đôi khi là một thói quen kì quặc hay cách người đó phản ứng lại với các tác động đến từ thế giới xung quanh. Mặc dù tính cách được hình thành và thay đổi thông qua quá trình học hỏi, phát triển của bản thân trong suốt quá trình sống nhưng với một số người, một số loại tính cách dường như gắn liền với họ bởi nhiều lý do như gặp các vấn đề về sức khỏe hay sự bảo thủ và cố chấp trong việc thay đổi tính cách.<br>**Metadata:** {}<br>

**Node ID:** 0ca78880-3b22-418f-b278-2c9e6ccc58da<br>**Similarity:** 2.2099711894989014<br>**Text:** Các loại bệnh tiểu đường.
Hiện nay, bệnh tiểu đường được phân thành ba loại chính, bao gồm: Bệnh tiểu đường loại 1 : Đây là một bệnh rối loạn tự miễn, nghĩa là hệ thống miễn dịch của cơ thể bị nhầm lẫn và tấn công chính tuyến tụy. Điều này làm cản trở quá trình sản xuất insulin và dẫn đến thiếu hụt đáng kể số lượng insulin cần thiết cho các hoạt động chuyển hóa glucose thành nguồn năng lượng của cơ thể, từ đó khiến cho lượng đường trong máu tăng cao. Bệnh tiểu đường loại 1 thường khá hiếm gặp, và xảy ra chủ yếu ở trẻ nhỏ. Bệnh tiểu đường loại 2 : Đây là loại tiểu đường thường gặp nhất hiện nay, ảnh hưởng lớn đến những người bị béo phì. Khi mắc bệnh tiểu đường loại 2, cơ thể sẽ bị đề kháng với insulin. Mặc dù hormone insulin vẫn được tiết ra bởi tuyến tụy, nhưng nó không thể chuyển hóa được đường glucose, khiến đường tích tụ dần trong máu. Hầu hết các trường hợp bị tiểu đường đều mắc phải loại tiểu đường này. Bệnh tiểu đường thai kỳ : Thường xảy ra đối với phụ nữ đang mang thai. Nhìn...<br>**Metadata:** {}<br>

**Node ID:** ef79e71a-d53b-4b1a-9a4f-e21eb32d00bf<br>**Similarity:** 2.1640465259552<br>**Text:** Các khái niệm liên quan về bệnh xốp thận, hoại tử gai thận.
Thận có hình hạt đậu, màu nâu đỏ và có bề mặt trơn láng. Thận có chiều dài khoảng 12cm, rộng 6cm, dày 3cm và nặng khoảng 130g. 1.1 Bệnh xốp thận là gì? Bệnh xốp thận (Cacchi-Ricci) là một rối loạn bẩm sinh của thận, đặc trưng bởi tình trạng giãn nở các ống dẫn nước tiểu nhỏ ở 1 hoặc cả 2 thận. Trong thận của bệnh nhân bị xốp thận có các nang và có thể tồn tại sỏi trong các nang này. Các nang này tạo thành dạng xốp như miếng bọt biển, giữ nước tiểu chảy tự do trong các ống và trong thận. Người mắc bệnh xốp thận có nguy cơ cao bị sỏi thận và nhiễm trùng đường tiểu do ứ đọng nước tiểu. Bệnh nhân thường phải lọc sỏi thận mỗi năm. Dù vậy, bệnh xốp thận vẫn được coi là một rối loạn lành tính, có tỷ lệ mắc bệnh khá thấp. Về triệu chứng, ở giai đoạn đầu bệnh xốp thận có thể không có triệu chứng. Dấu hiệu đầu tiên mà người bệnh thường gặp là nhiễm trùng đường tiểu hoặc sỏi thận . Các triệu chứng gây ra bởi 2 căn bệnh này gồm: Đau ở ...<br>**Metadata:** {}<br>

**Node ID:** cedf77bf-9982-478d-9b0b-b19674f1cc18<br>**Similarity:** 2.1548078060150146<br>**Text:** Tiêu chuẩn cận lâm sàng.
Tiểu bạch cầu : có ít nhất một trong các tiêu chuẩn sau: Que nhúng leucocyte esterase/ nước tiểu (+)Soi tươi nước tiểu đếm được ≥ 10 bạch cầu /mm3 NT.Sau khi quay ly tâm đếm được > 10 bạch cầu/μl NT. Que nhúng leucocyte esterase/ nước tiểu (+) Soi tươi nước tiểu đếm được ≥ 10 bạch cầu /mm3 NT. Sau khi quay ly tâm đếm được > 10 bạch cầu/μl NT. Tiểu vi khuẩn : có ít nhất 1 trong các tiêu chuẩn sau: Que nhúng diptistick nitrite (+).Nhuộm gram nước tiểu soi tươi/quang trường kính dầu (+).Cấy nước tiểu (+). Que nhúng diptistick nitrite (+). Nhuộm gram nước tiểu soi tươi/quang trường kính dầu (+). Cấy nước tiểu (+). Cấy nước tiểu được xem là (+) khi có ≥ 105 vi khuẩn /ml nước tiểu giữa dòng ở nữ hay 104 vi khuẩn /ml nước tiểu giữa dòng ở nam hoặc lấy qua ống thẳng ở nữ. Tuy nhiên, trong nhiễm khuẩn đường tiết niệu có tắc nghẽn, nếu sự tắc nghẽn này là hoàn toàn và thận đối diện không có nhiễm khuẩn, kết quả cấy vi khuẩn hay soi tìm bạch cầu thường (-), điều này gâ...<br>**Metadata:** {}<br>

**Node ID:** d30fe3d1-01d8-4277-a6d9-10298fa76b74<br>**Similarity:** 2.079346179962158<br>**Text:** Kiểm tra định kỳ sau Phẫu thuật Laser holmium.
Người bệnh cần được tái khám kiểm tra sau Phẫu thuật Laser holmium nhiều tuần đến nhiều tháng theo chỉ định của bác sĩ. Trong quá trình kiểm tra, bác sĩ sẽ đánh giá mức độ cải thiện triệu chứng tiểu tiện và khả năng tống được hết nước tiểu của bàng quang. Người bệnh cũng sẽ được chỉ định thực hiện xét nghiệm nước tiểu sàng lọc nhiễm trùng đường niệu và siêu âm hệ tiết niệu có đánh giá tồn dư nước tiểu sau đi tiểu cùng với đó là định lượng PSA . Giai đoạn sau đó, người bệnh chỉ cần tái khám bác sĩ tiết niệu 01 lần/ năm sau mổ. Nhiều năm sau các mô tuyến tiền liệt có thể phát triển trở lại, người bệnh có thể tái phát các triệu chứng như trước mổ, tuy nhiên những trường hợp này hiếm gặp.<br>**Metadata:** {}<br>

**Node ID:** c13aaaf1-d2a0-4873-b5d7-ccfae2b53048<br>**Similarity:** 2.0634143352508545<br>**Text:** Nhiễm toan ceton do tiểu đường.
Nhiễm toan ceton do tiểu đường là một tình trạng cấp cứu khi cơ thể thiếu insulin làm cho gan phải phân hủy chất béo thành ceton để tạo năng lượng. Sự tích tụ của ceton trong máu có thể làm thay đổi các phản ứng hóa học và đầu độc cơ thể khiến người bệnh rơi vào trạng thái hôn mê. Nhiễm toan ceton là biến chứng phổ biến nhất của bệnh tiểu đường tuýp 1, nhưng cũng có thể xảy ra với bệnh tiểu đường tuýp 2 và tiểu đường thai kỳ do các nguyên nhân sau: Không tiêm đủ insulin Ăn không đủ Có phản ứng insulin (lượng đường trong máu thấp) khi ngủ Người bị ốm hoặc bị nhiễm trùng, căng thẳng lớn gây đau tim và tác dụng của một số loại thuốc Các triệu chứng ban đầu của nhiễm toan ceton là: Khát nước Khô miệng Đi tiểu thường xuyên Các triệu chứng nghiêm trọng hơn của nhiễm toan ceton gồm có: Mệt mỏi Da khô hoặc đỏ bừng Hơi thở có mùi trái cây Buồn nôn, nôn hoặc đau bụng Khó thở Người bệnh thấy lo lắng, bối rối, bất tỉnh Nếu nghi ngờ người bệnh bị nhiễm toan ceton,...<br>**Metadata:** {}<br>

**Node ID:** 40e99791-64da-4aeb-9d5c-c2062de6148b<br>**Similarity:** 1.9866828918457031<br>**Text:** Vitamin tan trong chất béo và vitamin tan trong nước.
13 loại vitamin được biết đến được chia thành 2 loại gồm tan trong chất béo và tan trong nước. 1.1 Vitamin tan trong nước Các vitamin tan trong nước dễ dàng đào thải ra khỏi cơ thể và không dễ lưu trữ lại trong các mô. Có nhiều vitamin tan trong nước hơn so với những vitamin tan trong chất béo. Các vitamin tan trong nước bao gồm vitamin C, cộng với 8 vitamin B: Vitamin B1 (thiamine) Vitamin B2 (riboflavin) Vitamin B3 (niacin) Vitamin B5 (axit pantothenic) Vitamin B6 (pyridoxine) Vitamin B7 (biotin) Vitamin B9 (folate) Vitamin B12 (cobalamin) Vì các vitamin tan trong nước không được lưu trữ trong cơ thể nhưng được bài tiết qua nước tiểu, nên chúng ít có khả năng gây ra vấn đề ngay cả khi dùng ở liều cao. Tuy nhiên, dùng liều rất cao (megadoses) một số vitamin tan trong nước có thể dẫn đến các tác dụng phụ nguy hiểm tiềm tàng. Ví dụ, dùng vitamin B6 liều cao có thể dẫn đến tổn thương thần kinh không có khả năng phục hồi theo thời g...<br>**Metadata:** {}<br>

## hybrid retriever

Create a new search pipeline with score normalization and weighted harmonic mean combination.

```
PUT /_search/pipeline/hybrid-search-pipeline
{
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "harmonic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
```

In [35]:
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilter
from llama_index.core.vector_stores.types import VectorStoreQueryMode

dim = 768

client = OpensearchVectorClient(
    endpoint=os.getenv('OPENSEARCH_ENDPOINT', 'http://localhost:9200'),
    index=os.getenv('OPENSEARCH_INDEX', 'index-demo'),
    dim=dim,
    search_pipeline="hybrid-search-pipeline",
)
vector_store = OpensearchVectorStore(client)

logger.info(f"Loading Storage Context from {storage_context_fp}...")
docstore = SimpleDocumentStore.from_persist_dir(persist_dir=storage_context_fp)
storage_context = StorageContext.from_defaults(
    docstore=docstore,
    vector_store=vector_store
)
nodes = list(docstore.docs.values())

index = VectorStoreIndex(
    nodes, storage_context=storage_context, embed_model=embed_model
)
retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.HYBRID
)

2024-11-17 02:12:53.206 | INFO     | __main__:<module>:14 - Loading Storage Context from data/storage_context...


In [37]:
results = retriever.retrieve("bệnh tiểu đường?")
for result in results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

**Node ID:** 0ca78880-3b22-418f-b278-2c9e6ccc58da<br>**Similarity:** 1.0<br>**Text:** Các loại bệnh tiểu đường.
Hiện nay, bệnh tiểu đường được phân thành ba loại chính, bao gồm: Bệnh tiểu đường loại 1 : Đây là một bệnh rối loạn tự miễn, nghĩa là hệ thống miễn dịch của cơ thể bị nhầm lẫn và tấn công chính tuyến tụy. Điều này làm cản trở quá trình sản xuất insulin và dẫn đến thiếu hụt đáng kể số lượng insulin cần thiết cho các hoạt động chuyển hóa glucose thành nguồn năng lượng của cơ thể, từ đó khiến cho lượng đường trong máu tăng cao. Bệnh tiểu đường loại 1 thường khá hiếm gặp, và xảy ra chủ yếu ở trẻ nhỏ. Bệnh tiểu đường loại 2 : Đây là loại tiểu đường thường gặp nhất hiện nay, ảnh hưởng lớn đến những người bị béo phì. Khi mắc bệnh tiểu đường loại 2, cơ thể sẽ bị đề kháng với insulin. Mặc dù hormone insulin vẫn được tiết ra bởi tuyến tụy, nhưng nó không thể chuyển hóa được đường glucose, khiến đường tích tụ dần trong máu. Hầu hết các trường hợp bị tiểu đường đều mắc phải loại tiểu đường này. Bệnh tiểu đường thai kỳ : Thường xảy ra đối với phụ nữ đang mang thai. Nhìn...<br>**Metadata:** {}<br>

**Node ID:** 309fb63d-648f-40a1-ab8f-b63c19126e31<br>**Similarity:** 0.001<br>**Text:** Nguyên nhân và triệu chứng gây nên tiền đái tháo đường.
Insulin - hormone được tạo ra bởi tuyến tụy giúp cơ thể bạn chuyển hóa glucose thành nhiên liệu. Khi ăn thức ăn, carbohydrate trong thức ăn của bạn được chuyển hóa thành glucose , hay còn gọi là đường huyết. Glucose đó sẽ ở trong máu cho đến khi tuyến tụy tiết ra insulin. Insulin hoạt động giống như một chiếc chìa khóa mở các tế bào để cho phép glucose đi vào, nơi nó được sử dụng để cung cấp năng lượng cho cơ thể. Nếu không có insulin hoặc khi insulin không hoạt động hiệu quả, lượng glucose đó sẽ bị mắc kẹt trong máu và tích tụ, khiến lượng đường trong máu tăng lên. Trong bệnh đái tháo đường loại 2, tuyến tụy của bạn vẫn có thể sản xuất insulin, nhưng insulin đó dần trở nên kém hiệu quả hơn trong việc giúp glucose vào tế bào của bạn. Tiền đái tháo đường là một dấu hiệu cho thấy cơ thể bạn đang bắt đầu ngừng sử dụng insulin một cách hiệu quả nhất. Nói cách khác, cơ thể đang bắt đầu đề kháng với insulin. Khi cơ thể chống lại insu...<br>**Metadata:** {}<br>